In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import findspark 
findspark.init()
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import udf
from pyspark.sql.types import *

spark = SparkSession.builder.appName("Spark ML example on titanic data").getOrCreate()


In [3]:
path_and_file = './data_japan_house/13_Tokyo_20161_20182.csv'
sdf_tokyo = spark.read.csv(path_and_file,header='True',inferSchema='True',encoding='SHIFT-JIS')
sdf_tokyo.show(1)

+---+---------+---+-------+-----+-----+---+------+------+---------+----+----+---+------+-----+---+----+-----+-----+---------+-------+-------+-------+-------+----+----+---+----------+----+------+
| No|       種類| 地域|市区町村コード|都道府県名|市区町村名|地区名|最寄駅：名称|最寄駅：距離|    取引価格9| 坪単価| 間取り| 面積|取引価格13|土地の形状| 間口|延床面積|  建築年|建物の構造|       用途|今後の利用目的|前面道路：方位|前面道路：種類|前面道路：幅員|都市計画|建ぺい率|容積率|      取引時点|  改装|取引の事情等|
+---+---------+---+-------+-----+-----+---+------+------+---------+----+----+---+------+-----+---+----+-----+-----+---------+-------+-------+-------+-------+----+----+---+----------+----+------+
|  1|宅地(土地と建物)|商業地|  13101|  東京都| 千代田区|飯田橋|   飯田橋|     3|250000000|null|null| 80|  null| ほぼ台形|6.8| 330|昭和61年|   ＲＣ|住宅、事務所、店舗|    事務所|     南西|     区道|    8.0|商業地域|  80|500|平成30年第２四半期|null|  null|
+---+---------+---+-------+-----+-----+---+------+------+---------+----+----+---+------+-----+---+----+-----+-----+---------+-------+-------+-------+-------+----+----+---+----------+----+------+
only showing top 1 row



In [4]:
path_and_file = './data_japan_house/14_Kanagawa Prefecture_20161_20182.csv'
sdf_kanagawa = spark.read.csv(path_and_file,header='True',inferSchema='True',encoding='SHIFT-JIS')
sdf_kanagawa = sdf_kanagawa.filter(col('市区町村名').contains('横浜')|col('市区町村名').contains('川崎'))
sdf_kanagawa.show()

+---+---------+----+-------+-----+------+---+------+------+--------+------+------+---+------+-----+----+----+-----+-----+----+-------+-------+-------+-------+-----+----+---+----------+----+------+
| No|       種類|  地域|市区町村コード|都道府県名| 市区町村名|地区名|最寄駅：名称|最寄駅：距離|   取引価格9|   坪単価|   間取り| 面積|取引価格13|土地の形状|  間口|延床面積|  建築年|建物の構造|  用途|今後の利用目的|前面道路：方位|前面道路：種類|前面道路：幅員| 都市計画|建ぺい率|容積率|      取引時点|  改装|取引の事情等|
+---+---------+----+-------+-----+------+---+------+------+--------+------+------+---+------+-----+----+----+-----+-----+----+-------+-------+-------+-------+-----+----+---+----------+----+------+
|  1|宅地(土地と建物)| 住宅地|  14101| 神奈川県|横浜市鶴見区|朝日町|    浅野|    11|36000000|  null|  null| 60|  null|  長方形| 5.5|  95|平成29年|   木造|  住宅|     住宅|      南|     私道|    4.2|準工業地域|  60|200|平成29年第４四半期|null|  null|
|  2|宅地(土地と建物)| 住宅地|  14101| 神奈川県|横浜市鶴見区|朝日町|    浅野|    11|31000000|  null|  null| 50|  null|  長方形| 3.6|  85|平成29年|   木造|  住宅|     住宅|      東|     市道|    4.2|準工業地域|  60|200|平成29年第３四半期|null|  null|
|  3| 中古マンション等|

In [5]:
sdf = sdf_tokyo.unionAll(sdf_kanagawa)

In [6]:
describe_result = sdf.describe().toPandas()
describe_result

,summary,No,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員,都市計画,建ぺい率,容積率,取引時点,改装,取引の事情等
0,count,103922,103922,55002,103922,103922,103922,103922,103554,103435,...,94732,55000,54450,54156,103261,102762,102762,103922,43075,6503
1,mean,31192.466138065087,None,None,13409.291410865842,None,None,None,None,9.82792388629528,...,None,None,None,6.157563335549175,None,61.808450594577764,251.76650902084427,None,None,None
2,stddev,21647.94637294824,None,None,437.6623099845058,None,None,None,None,6.1965671774261635,...,None,None,None,4.548122470310611,None,12.944754147308164,157.21948728536037,None,None,None
3,min,1,中古マンション等,住宅地,13101,東京都,あきる野市,あかね台,あざみ野,0,...,その他,北,区画街路,1.0,商業地域,30,50,平成28年第１四半期,改装済,その他事情有り
4,max,75233,農地,工業地,14137,神奈川県,青梅市,（大字なし）,黒川(神奈川),9,...,店舗,西,都道,90.0,都市計画区域外,80,1300,平成30年第２四半期,未改装,隣地の購入、関係者間取引


In [7]:
sdf.printSchema()

root
 |-- No: integer (nullable = true)
 |-- 種類: string (nullable = true)
 |-- 地域: string (nullable = true)
 |-- 市区町村コード: integer (nullable = true)
 |-- 都道府県名: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 地区名: string (nullable = true)
 |-- 最寄駅：名称: string (nullable = true)
 |-- 最寄駅：距離: string (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 坪単価: integer (nullable = true)
 |-- 間取り: string (nullable = true)
 |-- 面積: string (nullable = true)
 |-- 取引価格13: integer (nullable = true)
 |-- 土地の形状: string (nullable = true)
 |-- 間口: string (nullable = true)
 |-- 延床面積: string (nullable = true)
 |-- 建築年: string (nullable = true)
 |-- 建物の構造: string (nullable = true)
 |-- 用途: string (nullable = true)
 |-- 今後の利用目的: string (nullable = true)
 |-- 前面道路：方位: string (nullable = true)
 |-- 前面道路：種類: string (nullable = true)
 |-- 前面道路：幅員: double (nullable = true)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 取引時点:

全部関係あるが、重要じゃなさそうな奴を削除

In [8]:
sdf = sdf.drop( 'No','地域','市区町村コード',
                     '地区名',
    '坪単価','土地の形状','間口','用途','今後の利用目的',
                     '前面道路：方位',
                     '前面道路：種類',
                     '前面道路：幅員',
                     '取引時点',
               '取引価格13',
    '改装',
                     '取引の事情等')

欠損値処理  

欠損値確認

In [9]:
sdf.count()

103922

In [10]:
sdf.describe().toPandas()

,summary,種類,都道府県名,市区町村名,最寄駅：名称,最寄駅：距離,取引価格9,間取り,面積,延床面積,建築年,建物の構造,都市計画,建ぺい率,容積率
0,count,103922,103922,103922,103554,103435,103922,46848,103922,36294,83196,83326,103261,102762,102762
1,mean,None,None,None,None,9.82792388629528,6.041636920767499E7,None,111.29266503903423,150.7402683819452,None,None,None,61.808450594577764,251.76650902084427
2,stddev,None,None,None,None,6.1965671774261635,2.4223598810029456E8,None,153.21097015129263,187.96804853242497,None,None,None,12.944754147308164,157.21948728536037
3,min,中古マンション等,東京都,あきる野市,あざみ野,0,1000,オープンフロア,10,10,平成10年,ブロック造,商業地域,30,50
4,max,農地,神奈川県,青梅市,黒川(神奈川),9,30000000000,６ＬＤＫ＋Ｓ,990,990,昭和63年,ＳＲＣ、ＲＣ,都市計画区域外,80,1300


ないものを未知に  

In [11]:
sdf = sdf.filter(
    col('最寄駅：名称').isNotNull()&
    col('最寄駅：距離').isNotNull()&
    col('都市計画').isNotNull()&
    col('建ぺい率').isNotNull()&
    col('容積率').isNotNull()&
    col('建築年').isNotNull()
    
)
sdf.count()

82457

In [12]:
for c in sdf.columns:
    print(c,'の欠損値の数',sdf.filter(col(c).isNull()).count())

種類 の欠損値の数 0
都道府県名 の欠損値の数 0
市区町村名 の欠損値の数 0
最寄駅：名称 の欠損値の数 0
最寄駅：距離 の欠損値の数 0
取引価格9 の欠損値の数 0
間取り の欠損値の数 37353
面積 の欠損値の数 0
延床面積 の欠損値の数 47254
建築年 の欠損値の数 0
建物の構造 の欠損値の数 1446
都市計画 の欠損値の数 0
建ぺい率 の欠損値の数 0
容積率 の欠損値の数 0


In [13]:

features_to_fill =[
    '間取り',
    '延床面積',
    '建築年',
    '建物の構造']

sdf = sdf.fillna('未知',subset=features_to_fill)





In [14]:
sdf.select('最寄駅：名称').show()

+------+
|最寄駅：名称|
+------+
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
|   飯田橋|
+------+
only showing top 20 rows



In [15]:
sdf.groupby('最寄駅：距離').count().show()

+-------+-----+
| 最寄駅：距離|count|
+-------+-----+
|      7| 4776|
|     15| 2703|
|     11| 4351|
|1H30?2H|   27|
|     29|  468|
|      3| 5188|
|      8| 6170|
|     28|  577|
|     22|   89|
|     16| 2292|
|      0|  329|
|      5| 5820|
|     18| 1846|
|     27|   21|
|     17|  260|
|     26|  604|
|      6| 6558|
|     19| 1581|
|     23|  951|
|     25|  829|
+-------+-----+
only showing top 20 rows



In [16]:
li = ['1H30?2H','1H?1H30','30分?60分','2H?']
sdf = sdf.filter(~(col('最寄駅：距離').isin(li)))

In [17]:
sdf.filter(col('最寄駅：距離').isNull()).count()

0

駅までの距離を解決

間取りにはnullが入っている  
面積で補間したい

In [18]:
sdf.select('面積').describe().show()

+-------+------------------+
|summary|                面積|
+-------+------------------+
|  count|             78947|
|   mean| 88.49092730431254|
| stddev|105.63867365237994|
|    min|                10|
|    max|               990|
+-------+------------------+



In [19]:
sdf = sdf.filter(~(col('面積').contains('以上')))
sdf = sdf.filter(~(col('延床面積').contains('以上')))


500以上は広すぎ、

In [20]:
sdf = sdf.filter(~(col('面積')>500))

残りも、わからないのが未知

In [21]:
sdf.show()

+---------+-----+-----+------+------+---------+----+---+----+-----+-----+----+----+---+
|       種類|都道府県名|市区町村名|最寄駅：名称|最寄駅：距離|    取引価格9| 間取り| 面積|延床面積|  建築年|建物の構造|都市計画|建ぺい率|容積率|
+---------+-----+-----+------+------+---------+----+---+----+-----+-----+----+----+---+
|宅地(土地と建物)|  東京都| 千代田区|   飯田橋|     3|250000000|  未知| 80| 330|昭和61年|   ＲＣ|商業地域|  80|500|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 22000000|  １Ｋ| 30|  未知|昭和60年|  ＳＲＣ|商業地域|  80|700|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     1| 69000000|２ＬＤＫ| 70|  未知|昭和59年|  ＳＲＣ|商業地域|  80|600|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 21000000|  １Ｋ| 25|  未知|昭和60年|  ＳＲＣ|商業地域|  80|700|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     4| 45000000|１ＬＤＫ| 45|  未知|昭和57年|  ＳＲＣ|商業地域|  80|700|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     1| 66000000|２ＬＤＫ| 55|  未知|昭和59年|  ＳＲＣ|商業地域|  80|600|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     5| 19000000|  １Ｋ| 20|  未知|平成15年|   ＲＣ|商業地域|  80|500|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 37000000|１ＬＤＫ| 45|  未知|平成24年|   ＲＣ|商業地域|  80|500|
| 中古マンション等|  東京都| 千代田区|   飯田橋|  

In [22]:
sdf = sdf.filter(col('建築年').isNotNull())
sdf = sdf.filter(~(col('建築年').contains('戦前')))

In [23]:
sdf.select('建築年').collect()

[Row(建築年='昭和61年'),
 Row(建築年='昭和60年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和60年'),
 Row(建築年='昭和57年'),
 Row(建築年='昭和59年'),
 Row(建築年='平成15年'),
 Row(建築年='平成24年'),
 Row(建築年='平成15年'),
 Row(建築年='昭和61年'),
 Row(建築年='昭和59年'),
 Row(建築年='平成11年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和60年'),
 Row(建築年='平成19年'),
 Row(建築年='平成10年'),
 Row(建築年='平成19年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和59年'),
 Row(建築年='昭和57年'),
 Row(建築年='昭和45年'),
 Row(建築年='平成19年'),
 Row(建築年='昭和57年'),
 Row(建築年='平成19年'),
 Row(建築年='平成9年'),
 Row(建築年='平成15年'),
 Row(建築年='平成16年'),
 Row(建築年='平成16年'),
 Row(建築年='平成16年'),
 Row(建築年='平成15年'),
 Row(建築年='平成5年'),
 Row(建築年='平成24年'),
 Row(建築年='平成24年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成13年'),
 Row(建築年='昭和52年'),
 Row(建築年='昭和51年'),
 Row(建築年='昭和47年'),
 Row(建築年='昭和59年'),
 Row(建築年='平成13年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成25年'),
 Row(建築年='平成16年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成13年'),
 Row(建築年='昭和47年'),
 Row(建築年='平成25年'),
 Row(建築年='昭和52年'),
 Row(建築年='昭和51年'),
 Row(建築年='昭和45年'),
 Row(建築年='平成13

In [24]:
import re
def year_transform(year):
    if re.match('昭和元年', year):
        year = 2018-1926
        return year
    elif re.match('昭和',year):
        year = 2018-(1925 + int(re.sub(r'\D','',year)))
        return year
    elif re.match('平成元年', year):
        year = 2018-1989
        return year
    elif re.match('平成',year):
        year = 2018-(1988 + int(re.sub(r'\D','',year)))
        return year
    else:
        return year

In [25]:
year_series = np.array([])
for i in sdf.select('建築年').collect():
    print(year_transform(i[0]))
    np.append(year_series,year_transform(i[0]))
year_series

32
33
34
33
36
34
15
6
15
32
34
19
34
34
33
11
20
11
34
34
34
36
48
11
36
11
21
15
14
14
14
15
25
6
6
46
17
41
42
46
34
17
46
5
14
46
17
46
5
41
42
48
17
42
48
19
39
42
39
14
34
12
16
57
14
11
14
14
12
14
14
12
15
13
12
12
12
14
14
14
12
12
12
12
31
5
14
18
12
12
12
15
12
14
13
12
14
12
14
18
12
12
12
12
12
14
12
12
13
12
12
55
14
5
11
16
16
16
49
12
16
12
16
5
5
16
5
16
5
48
4
12
5
5
4
12
5
6
15
15
46
15
34
15
15
15
13
37
16
16
16
13
16
11
11
46
11
4
39
13
13
13
39
13
39
39
39
45
18
13
17
39
40
48
15
23
46
36
28
15
5
15
15
15
30
15
5
15
37
5
18
18
14
13
5
18
14
17
17
17
4
8
38
38
38
38
38
38
38
34
38
38
34
36
36
18
1
14
19
2
21
54
29
55
6
6
45
15
10
18
17
38
15
10
10
15
15
15
18
17
6
20
7
40
21
7
18
20
20
18
7
18
40
34
34
36
35
3
13
28
28
43
34
13
13
3
16
11
31
42
17
15
11
11
3
15
14
15
15
5
5
5
5
2
14
36
14
2
33
14
14
30
14
56
14
14
14
14
44
44
13
2
16
13
13
13
13
10
13
10
49
18
13
41
41
41
41
43
29
43
43
41
39
43
14
14
11
13
11
13
11
11
11
11
14
14
13
14
11
11
11
11
46
11
13
11
14
1

38
14
6
47
64
45
6
15
7
59
51
50
46
38
6
14
6
7
41
44
6
18
35
47
6
33
6
12
12
12
12
12
12
12
12
12
12
12
12
12
17
17
39
38
20
38
13
38
13
38
38
15
38
4
38
38
38
38
15
39
34
14
38
15
18
38
30
30
4
14
12
15
15
12
16
21
2
53
39
53
1
25
17
39
11
19
14
14
19
20
14
12
46
14
14
39
46
14
15
39
40
39
2
12
12
40
14
14
16
14
18
17
18
14
39
2
15
31
14
33
15
15
15
15
15
10
19
15
48
48
15
18
45
47
10
14
14
14
20
48
10
15
45
45
15
47
47
18
15
49
47
47
48
47
47
47
15
15
47
47
15
15
47
47
47
14
47
47
10
47
47
15
7
47
47
47
16
47
47
47
48
47
15
15
15
47
15
47
47
47
15
37
47
38
47
47
47
47
47
47
47
47
45
34
8
13
15
13
13
6
12
12
13
20
20
13
7
7
6
13
6
13
8
13
13
15
13
7
13
13
13
15
40
15
13
13
40
18
13
14
15
13
8
14
40
13
18
13
13
13
15
17
18
17
13
13
23
20
13
6
13
10
12
6
4
4
5
5
15
10
10
15
15
10
12
10
10
15
26
44
11
11
11
11
24
11
51
3
4
4
37
42
40
13
42
19
12
19
42
18
22
18
19
18
30
4
40
15
4
16
30
22
18
6
31
22
34
4
4
6
4
14
18
22
15
18
42
18
18
6
19
5
4
18
15
42
15
15
15
18
5
5
15
6
15
16
20
53
19


2
2
1
20
39
25
25
20
33
20
30
15
30
20
28
12
15
3
32
32
3
32
26
26
34
13
25
15
17
19
47
58
17
21
1
2
15
6
17
50
15
15
17
12
3
3
3
8
3
65
47
9
48
47
47
9
19
17
3
58
17
9
8
8
9
17
20
8
50
47
17
4
8
8
8
4
8
10
62
16
17
13
18
18
19
16
40
6
41
40
13
13
17
17
18
18
17
17
17
17
17
44
17
23
18
47
1
51
18
18
19
1
55
50
47
58
5
23
13
5
58
30
30
34
30
30
30
15
30
30
10
15
4
23
21
9
30
14
34
10
10
4
4
4
4
4
30
20
4
4
4
2
24
21
14
20
15
28
16
15
30
30
23
9
9
9
9
9
9
9
9
32
26
14
14
15
14
15
15
15
15
19
50
7
14
11
14
15
15
14
7
15
15
12
12
12
12
17
20
12
20
23
21
21
5
12
17
12
16
21
17
17
16
8
28
8
4
11
26
40
5
28
12
10
24
40
12
10
40
32
56
33
33
14
28
58
10
21
6
14
44
6
21
6
14
12
6
14
10
10
58
6
10
21
2
21
21
20
40
13
50
5
14
14
30
14
14
32
49
7
14
14
38
30
14
14
14
5
20
58
29
63
19
15
19
17
17
17
17
17
17
17
17
7
24
33
63
11
18
12
13
26
13
12
52
33
21
12
8
21
20
6
1
20
24
8
21
14
8
8
20
18
34
39
39
15
34
32
15
15
15
4
20
15
3
40
34
14
48
11
11
1
34
15
40
13
13
14
11
11
20
39
38
3
11
13
38
40
38
4

14
4
46
15
33
38
46
6
17
7
15
46
2
2
46
46
9
6
6
28
9
22
11
35
11
18
1
1
38
1
4
12
1
1
5
33
11
27
22
18
18
23
16
11
12
22
22
0
18
12
18
12
2
39
18
18
3
18
40
57
18
39
49
18
3
18
13
16
18
21
18
32
3
40
3
21
3
3
18
3
3
16
54
27
3
51
27
6
51
49
1
35
3
3
38
8
8
8
4
13
29
16
63
16
4
47
29
15
3
44
3
38
3
29
29
3
8
11
19
13
8
8
11
13
19
19
27
26
25
19
19
31
32
11
13
28
2
16
25
16
60
11
49
11
32
11
49
10
10
9
10
11
31
12
29
29
50
33
35
35
48
48
21
29
6
40
15
11
18
15
41
41
26
20
26
16
13
9
10
47
28
16
16
33
7
16
6
16
7
7
17
10
5
21
9
7
15
9
45
60
15
9
50
45
9
10
30
15
33
9
9
14
9
9
9
39
13
14
13
21
6
11
13
30
6
4
11
4
71
13
10
10
27
11
22
30
11
8
39
33
13
51
4
28
4
53
21
8
8
45
21
24
30
10
9
7
48
19
29
19
19
9
47
27
27
18
7
7
10
8
17
15
19
24
34
24
19
10
9
24
10
20
25
30
19
16
16
14
16
14
22
5
16
19
46
62
12
4
6
51
31
45
18
18
35
35
35
48
14
0
39
4
19
13
1
19
14
17
14
13
4
4
14
18
16
13
4
31
14
19
19
48
31
39
14
19
48
19
12
13
6
22
2
2
12
2
47
12
13
13
6
47
25
13
19
22
22
0
9
29
15
28
21
4
8
1

13
14
13
13
28
15
15
13
15
9
2
15
2
40
13
2
15
13
9
52
13
15
9
14
2
2
2
12
14
14
12
14
14
39
1
1
0
0
14
7
21
21
14
7
39
12
19
12
14
21
19
7
19
14
14
28
15
63
12
21
14
21
26
17
17
14
14
12
9
27
27
2
2
32
23
2
2
27
2
17
33
19
2
52
7
13
33
27
12
30
18
17
32
14
23
25
17
30
32
6
14
25
33
17
26
30
32
25
14
25
13
13
26
21
14
25
26
13
46
13
33
33
55
34
14
29
14
25
30
30
10
7
16
6
19
18
28
0
10
14
10
11
41
15
1
10
1
32
38
18
1
14
14
18
31
6
16
11
11
6
16
47
12
23
16
14
16
18
16
14
21
41
47
19
10
39
10
17
10
14
11
41
18
21
11
14
11
8
17
10
12
11
18
6
10
10
10
9
11
11
11
11
10
10
11
16
10
16
11
14
10
22
21
11
21
16
10
10
11
31
31
16
6
10
11
16
52
16
10
18
16
16
16
16
11
21
14
10
0
34
16
22
1
18
22
34
43
17
2
16
28
28
1
56
11
11
1
44
29
57
9
2
11
37
2
2
11
3
47
9
9
18
9
9
9
11
9
14
9
14
14
34
8
9
10
9
47
12
6
9
9
9
47
9
12
5
34
9
47
12
9
9
12
9
9
9
34
34
9
9
1
1
67
57
4
14
7
11
18
4
15
14
9
9
11
17
9
4
11
6
9
15
12
6
12
9
9
9
9
34
18
9
47
14
9
9
15
34
34
9
18
18
47
4
34
28
1
12
34
47
47
17
3
35
4


...

42
0
31
8
32
32
1
1
1
23
1
46
48
42
8
30
50
44
20
10
58
27
18
3
52
27
2
2
2
44
2
2
52
24
22
30
1
45
42
21
1
3
40
1
49
21
48
2
28
28
2
36
28
29
0
14
39
53
1
1
1
14
25
4
8
28
1
29
26
53
32
23
37
28
33
70
2
58
2
22
44
2
2
29
24
46
22
36
23
1
1
28
28
2
2
1
1
1
1
26
26
1
2
38
38
2
2
50
65
9
9
49
23
23
64
1
23
58
6
25
16
0
18
58
31
31
27
48
1
27
27
31
2
2
31
17
31
59
34
2
27
10
12
0
1
1
42
27
31
7
2
31
29
29
6
47
10
7
7
17
23
2
2
2
2
6
2
1
1
1
1
1
1
1
1
26
1
18
29
20
29
29
28
28
4
28
9
28
15
14
3
14
18
33
23
24
33
32
24
34
16
34
34
16
34
17
34
43
17
32
32
17
1
38
1
1
1
12
5
12
12
5
2
12
12
33
12
12
13
21
5
22
10
25
25
12
22
25
25
25
22
14
20
22
5
26
26
20
26
14
23
19
15
32
27
30
42
2
2
2
27
32
32
27
27
27
19
31
32
32
0
29
32
23
4
23
1
5
35
30
39
0
41
46
30
32
32
13
1
52
1
55
56
1
53
31
9
25
41
1
23
30
27
2
48
32
1
46
2
2
21
26
2
21
2
2
3
7
10
54
25
20
1
0
3
11
30
33
32
29
39
2
0
0
42
27
2
1
31
35
25
2
2
3
29
53
62
1
60
36
2
28
10
30
46
19
21
30
8
30
30
2
30
40
20
24
30
30
20
27
30
29
29
30
2

42
25
2
39
27
1
39
27
23
23
2
17
39
31
27
1
0
0
0
0
31
29
1
34
12
13
29
17
1
12
1
1
18
16
25
33
1
41
1
25
12
24
29
29
34
46
27
1
29
12
12
2
12
29
2
12
12
16
24
2
2
18
29
37
5
12
12
12
12
12
12
12
12
12
12
12
12
12
10
12
30
1
1
16
33
32
16
22
16
37
1
1
1
32
32
32
26
1
1
33
33
11
1
1
32
4
11
26
33
32
16
35
4
22
4
44
14
16
35
35
16
11
4
33
31
24
31
1
1
33
31
31
32
33
23
23
23
20
1
5
3
16
52
33
13
9
13
34
15
9
13
42
42
33
35
15
52
52
15
13
34
2
2
9
2
2
15
33
0
24
0
46
30
1
1
1
0
16
46
16
16
23
1
1
1
0
16
1
56
56
31
19
10
1
1
1
46
16
27
1
2
2
1
1
2
2
33
1
45
34
1
46
12
16
2
2
2
2
2
2
1
26
16
26
2
2
27
28
3
40
31
46
38
29
3
28
2
2
26
36
55
1
2
2
1
2
0
42
42
6
48
2
2
34
45
26
44
35
39
31
44
44
35
31
44
44
44
35
32
28
35
45
28
2
23
31
16
34
25
28
5
34
28
13
25
21
32
32
13
34
35
34
35
14
21
42
24
13
33
35
13
14
35
5
23
32
31
13
31
14
22
14
14
16
33
21
31
13
21
28
10
13
16
21
22
14
23
34
35
54
31
31
14
14
33
21
36
14
21
31
31
33
34
32
13
13
7
19
31
13
28
24
34
34
0
0
11
12
1
1
12
1
1
13
30
1
39


36
36
1
38
15
16
5
19
17
38
3
18
17
36
36
17
38
33
2
12
35
36
36
2
38
3
17
36
5
1
33
1
2
1
17
23
13
28
24
13
13
1
14
25
14
1
20
14
2
13
2
16
14
32
21
22
14
33
6
0
0
31
6
20
40
6
6
38
6
24
6
18
44
28
36
40
29
1
32
8
6
6
45
2
48
2
1
1
8
6
24
20
17
33
29
1
1
6
25
22
3
32
44
16
37
2
41
2
34
24
40
6
33
19
24
6
33
0
50
1
13
17
17
19
17
13
29
0
16
13
32
10
10
1
40
28
40
2
14
22
17
1
20
41
12
11
14
50
42
1
1
1
34
40
0
50
45
22
13
50
50
15
50
10
50
42
38
1
28
1
42
15
50
11
11
7
50
49
11
42
35
21
36
13
45
22
50
10
13
10
50
13
30
30
48
7
50
15
50
21
11
15
41
3
16
50
22
31
47
2
25
24
42
11
50
12
11
16
32
28
27
40
44
32
49
50
18
18
18
50
50
3
3
3
22
13
15
17
24
26
29
40
0
1
0
44
1
22
2
20
8
18
26
1
19
27
1
19
19
3
3
19
37
37
37
36
38
17
18
18
38
11
18
17
12
5
12
17
18
33
11
33
15
1
33
14
15
2
32
2
32
11
11
33
2
1
45
22
1
1
30
1
21
1
30
5
18
19
21
22
24
10
2
18
20
34
21
2
3
32
1
15
19
15
14
11
19
4
32
18
20
18
25
12
18
20
1
20
32
16
2
27
2
6
18
2
20
2
2
18
3
29
25
3
3
3
3
18
16
32
20
16
8
31
20
1
19

28
41
41
4
4
4
17
29
4
15
39
28
28
2
30
30
30
28
28
35
40
44
27
25
22
37
1
1
13
55
2
22
32
17
29
2
25
9
5
9
5
34
9
22
22
22
28
28
9
5
28
5
9
16
3
12
29
31
16
17
11
31
17
28
12
25
35
35
6
16
14
22
12
17
4
9
12
12
52
16
19
12
14
11
16
19
11
17
14
14
14
0
40
2
26
13
1
2
12
0
35
36
31
17
1
15
20
19
29
29
36
20
16
48
27
27
31
1
16
2
57
10
10
35
36
34
2
2
21
36
2
44
1
15
1
1
1
1
1
2
1
47
10
2
1
1
1
1
31
30
2
15
15
2
2
2
1
1
19
1
1
1
31
31
1
1
26
40
52
2
42
3
39
1
1
1
1
1
1
1
1
1
1
1
1
28
2
1
42
16
2
31
21
31
2
2
14
28
29
23
21
51
1
0
0
29
18
1
1
1
0
24
2
1
1
1
1
1
1
1
18
20
2
24
25
18
1
1
2
25
26
2
2
48
2
2
29
26
2
51
2
2
27
18
3
3
2
3
2
2
11
52
26
26
33
24
32
32
28
0
23
15
2
1
32
13
27
2
15
15
19
2
15
33
32
2
23
23
17
2
1
1
1
2
2
2
7
1
1
22
1
1
1
20
1
22
42
1
22
2
1
18
28
29
21
55
40
25
18
36
2
2
2
23
30
29
31
23
31
2
31
11
0
0
0
0
53
33
20
1
33
3
36
29
2
2
32
2
0
24
1
31
24
1
31
31
31
1
1
17
30
17
33
17
24
1
1
2
2
27
1
22
10
10
10
10
44
44
1
48
1
33
1
26
1
1
1
1
1
19
44
1
1
44
33
2
44
14
2

array([], dtype=float64)

In [26]:
udf_year_transform = udf(year_transform, IntegerType())
sdf = sdf.withColumn("年数", udf_year_transform("建築年"))
sdf = sdf.drop('建築年')
sdf.show(5)

+---------+-----+-----+------+------+---------+----+---+----+-----+----+----+---+---+
|       種類|都道府県名|市区町村名|最寄駅：名称|最寄駅：距離|    取引価格9| 間取り| 面積|延床面積|建物の構造|都市計画|建ぺい率|容積率| 年数|
+---------+-----+-----+------+------+---------+----+---+----+-----+----+----+---+---+
|宅地(土地と建物)|  東京都| 千代田区|   飯田橋|     3|250000000|  未知| 80| 330|   ＲＣ|商業地域|  80|500| 32|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 22000000|  １Ｋ| 30|  未知|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     1| 69000000|２ＬＤＫ| 70|  未知|  ＳＲＣ|商業地域|  80|600| 34|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 21000000|  １Ｋ| 25|  未知|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     4| 45000000|１ＬＤＫ| 45|  未知|  ＳＲＣ|商業地域|  80|700| 36|
+---------+-----+-----+------+------+---------+----+---+----+-----+----+----+---+---+
only showing top 5 rows



In [27]:
sdf = sdf.filter(col('容積率').isNotNull())
sdf = sdf.withColumn("延床面積", col('面積')*col('容積率'))

これで良さそうなデータがあります。 

pipelineなどを作って、一回予測してみましょう

その前に、もう一回スキーマを見ましょう

In [28]:
sdf.printSchema()

root
 |-- 種類: string (nullable = true)
 |-- 都道府県名: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 最寄駅：名称: string (nullable = true)
 |-- 最寄駅：距離: string (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 間取り: string (nullable = false)
 |-- 面積: string (nullable = true)
 |-- 延床面積: double (nullable = true)
 |-- 建物の構造: string (nullable = false)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 年数: integer (nullable = true)



型変換

In [29]:
sdf = sdf.withColumn("最寄駅：距離", sdf["最寄駅：距離"].cast(IntegerType()))
sdf = sdf.withColumn("面積", sdf["面積"].cast(IntegerType()))

In [30]:
sdf.printSchema()


root
 |-- 種類: string (nullable = true)
 |-- 都道府県名: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 最寄駅：名称: string (nullable = true)
 |-- 最寄駅：距離: integer (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 間取り: string (nullable = false)
 |-- 面積: integer (nullable = true)
 |-- 延床面積: double (nullable = true)
 |-- 建物の構造: string (nullable = false)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 年数: integer (nullable = true)



## ML

In [31]:
cols = sdf.columns
cols

['種類',
 '都道府県名',
 '市区町村名',
 '最寄駅：名称',
 '最寄駅：距離',
 '取引価格9',
 '間取り',
 '面積',
 '延床面積',
 '建物の構造',
 '都市計画',
 '建ぺい率',
 '容積率',
 '年数']

In [32]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
categoricalColumns = ['種類', '都道府県名', '市区町村名', '最寄駅：名称', '間取り', '建物の構造', '都市計画']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

numericCols = ['最寄駅：距離', '面積', '延床面積', '建ぺい率', '容積率', '年数']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [33]:
sdf.show(5)

+---------+-----+-----+------+------+---------+----+---+-------+-----+----+----+---+---+
|       種類|都道府県名|市区町村名|最寄駅：名称|最寄駅：距離|    取引価格9| 間取り| 面積|   延床面積|建物の構造|都市計画|建ぺい率|容積率| 年数|
+---------+-----+-----+------+------+---------+----+---+-------+-----+----+----+---+---+
|宅地(土地と建物)|  東京都| 千代田区|   飯田橋|     3|250000000|  未知| 80|40000.0|   ＲＣ|商業地域|  80|500| 32|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 22000000|  １Ｋ| 30|21000.0|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     1| 69000000|２ＬＤＫ| 70|42000.0|  ＳＲＣ|商業地域|  80|600| 34|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     3| 21000000|  １Ｋ| 25|17500.0|  ＳＲＣ|商業地域|  80|700| 33|
| 中古マンション等|  東京都| 千代田区|   飯田橋|     4| 45000000|１ＬＤＫ| 45|31500.0|  ＳＲＣ|商業地域|  80|700| 36|
+---------+-----+-----+------+------+---------+----+---+-------+-----+----+----+---+---+
only showing top 5 rows



In [34]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(sdf)
sdf = pipelineModel.transform(sdf)

In [35]:
selectedCols = ['features'] + cols
selectedCols

['features',
 '種類',
 '都道府県名',
 '市区町村名',
 '最寄駅：名称',
 '最寄駅：距離',
 '取引価格9',
 '間取り',
 '面積',
 '延床面積',
 '建物の構造',
 '都市計画',
 '建ぺい率',
 '容積率',
 '年数']

In [36]:
sdf = sdf.select(selectedCols)
sdf.printSchema()

root
 |-- features: vector (nullable = true)
 |-- 種類: string (nullable = true)
 |-- 都道府県名: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 最寄駅：名称: string (nullable = true)
 |-- 最寄駅：距離: integer (nullable = true)
 |-- 取引価格9: long (nullable = true)
 |-- 間取り: string (nullable = false)
 |-- 面積: integer (nullable = true)
 |-- 延床面積: double (nullable = true)
 |-- 建物の構造: string (nullable = false)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率: integer (nullable = true)
 |-- 容積率: integer (nullable = true)
 |-- 年数: integer (nullable = true)



In [37]:
train,test = sdf.randomSplit([0.7,0.3],seed = 0)
print('train:',train.count())

train: 54674


モデリング

In [38]:
from pyspark.ml.regression import RandomForestRegressor

dt = RandomForestRegressor(featuresCol = 'features',labelCol = '取引価格9')
dtModel = dt.fit(train)
dt_pred = dtModel.transform(test)


In [51]:
dt_pred.select("prediction","取引価格9").show()


+--------------------+--------+
|          prediction|   取引価格9|
+--------------------+--------+
|2.5317042735197585E7|19000000|
| 3.535634116173114E7|33000000|
|3.2904317761455752E7|43000000|
|4.2324383344918735E7|69000000|
| 3.671709783440156E7|28000000|
|3.2837414280101947E7|25000000|
|  6.25192453701724E7|31000000|
|3.3906148830650195E7|21000000|
| 5.962538660011089E7|65000000|
| 4.542111223869635E7|34000000|
| 3.599818037376322E7|39000000|
|2.1556938267318584E7|23000000|
|2.1556938267318584E7|15000000|
|2.1556938267318584E7|17000000|
|2.1556938267318584E7|21000000|
|2.1556938267318584E7|20000000|
| 2.190055681944332E7|23000000|
| 2.190055681944332E7|21000000|
|2.1556938267318584E7|20000000|
|2.1556938267318584E7|21000000|
+--------------------+--------+
only showing top 20 rows



In [52]:
evaluator = RegressionEvaluator(
    labelCol="取引価格9", predictionCol="prediction", metricName="rmse")

In [53]:
evaluator.evaluate(dt_pred)

60759342.29271417

結果として、あり得ない予測です。  失敗です。。。。。  
来週は改善しましょう